In [ ]:
import numpy as np
import matplotlib.pyplot as plt

image = np.load('../output/aligned/image_norm.npy')
albedo = cv2.imread('../output/albedo_maps/predicted_albedo.png', cv2.IMREAD_GRAYSCALE) / 255.0
R_model = np.load('../output/reflectance_models/expected_reflectance.npy')
init_dem = np.load('../output/aligned/dem_resized.npy')


In [ ]:
def compute_normals(dem, scale=1.0):
    dx = np.gradient(dem, axis=1) * scale
    dy = np.gradient(dem, axis=0) * scale
    dz = np.ones_like(dem)
    
    norm = np.sqrt(dx**2 + dy**2 + dz**2)
    return dx / norm, dy / norm, dz / norm  # Surface normal components



In [ ]:
def sfs_optimization(image, albedo, R_model, dem_init, iterations=50, alpha=0.1):
    dem = dem_init.copy()
    for it in range(iterations):
        dx, dy, dz = compute_normals(dem)
        slope_factor = dz  # Approximation: verticality = brightness proxy
        
        # Compute new reflectance (simplified)
        R_current = albedo * slope_factor
        
        # Difference from model
        diff = R_model - R_current
        
        # Update DEM using gradient feedback
        dem += alpha * diff  # alpha = learning rate
        if it % 10 == 0:
            print(f"Iteration {it}, mean diff: {np.mean(np.abs(diff)):.4f}")
    return dem



In [ ]:
dem_final = sfs_optimization(image, albedo, R_model, init_dem, iterations=100, alpha=0.05)

plt.imshow(dem_final, cmap='terrain')
plt.title("Optimized DEM")
plt.colorbar()
plt.show()

np.save('../output/sfs/dem_final.npy', dem_final)
cv2.imwrite('../output/sfs/dem_visual.png', (dem_final - np.min(dem_final)) / np.ptp(dem_final) * 255)
